# Tutorial

This repository provides an implementation of basic compositional semantics ; it can be used to check truth-value of sentences against models, type correctness, first-order formulas for truth conditions. The following provides an introduction.

## Basic functionality

Let's consider the following model:

  * Ahmed and Ben are the men
  * Rosa and Ren are the women
  * All but Ren smiles
  * Ahmed loves Rosa, who loves Ren, who loves Ahmed

Against this model, we want to find the truth-value of the following sentences:

  1. Ahmed smiles.
  1. Rosa loves Ahmed.
  1. Every man smiles.

This is done as follows:

In [2]:
# Let's first import our model (implemented by default in nonlogical.py ; see later to construct one's own models):
# This file also contains the non-logical vocab associated with the model
from nonlogical import *

# "every" is part of the logical vocab and needs its own import
from logical import every

# Finally let's set up the system to use the rules of composition in H&K.
from value import Value
from stdhk import StdHK # stdhk stands for "Standard Heim & Kratzer"
Value.rules = StdHK


# The meanings are lower-case named variables 
# They can be combined with "+"
sentence1 = ahmed + smiles
sentence2 = rosa + (loves + ahmed) # The parenthesis ensure that "Ahmed" is constituent at LF with "loves"
sentence3 = (every + man) + smiles

# The truth-values of the sentences against an empty assignment function can be obtained using the method val()
print("Sentence 1 is", sentence1.val())
print("Sentence 2 is", sentence2.val())
print("Sentence 3 is", sentence3.val())


Sentence 1 is True
Sentence 2 is False
Sentence 3 is True


The system can deal with assignment functions. Consider the following sentences/LFs:

 1. Ahmed loves pro<sub>1</sub>
 1. every man 1 Rosa loves t<sub>1</sub>

For these sentences, more logical vocabulary needs to be imported.

In [7]:
# Importing binders and pronouns/traces
from logical import pro, binder

# These items take a numerical index as an argument
sentence4 = ahmed + (loves + pro(1))
sentence5 = (every + man) + (binder(1) + (rosa + (loves + pro(1))))

# Sentence 4 cannot be evaluated against an empty assignment function
# Assignemnent functions are implemented as dictionaries
# They can be passed as arguments to val()
print("If g maps 1 to Rosa, sentence 4 is", sentence1.val({1 : "Rosa"}))

# SUBTLE POINT : can be ignored
# The assignment function should contain individuals (i.e. "Rosa"), not semantic values (i.e. rosa)
# This is because semantic values encode their dependency to contexts
# See later

# Sentence 5 can be evaluated in an empty context
print("Sentence 5 is", sentence5.val())



If g maps 1 to Rosa, sentence 4 is True
Sentence 5 is False


Type mismatch raises an exception. For instance, the following LF for instance:

  1. (Ahmed Ren) smiles

In [8]:
sentence6 = (ahmed + ren) + smiles

TypeMismatch: Types e and e can't combine.
 Rules are: [<stdhk.FA object at 0x000001C8F2F53470>, <stdhk.Abstraction object at 0x000001C8F2F534E0>, <stdhk.PredicateModification object at 0x000001C8F2F53518>]

## Advanced functionality

### Creating one's own lexical items

Lexical items have two components: a type and a value.